In [ ]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
import json
import folium

In [ ]:
df = pd.read_pickle("data/data.pkl")
df['Date'] = df['Votation'].map(lambda x : x.split(' ')[0])
df['Votation'] = df['Votation'].map(lambda x : ' '.join(x.split(' ')[1:]))
df.head()

In [ ]:
xls = pd.ExcelFile('data/Recommandations des partis.xls')
nb_sheets = len(xls.sheet_names)
recommand = pd.DataFrame()

for i in range ((2017 - 1981 + 1)) :
#for i in range (6,7) :
 
    x = xls.parse(i)
    x.columns = range(len(x.columns))
    
    base_nb = x[x.iloc[:, 0] == 'Parti 1)'].index[0]
    

    x = x.dropna(axis=1, how='all')

    recommand_inter = pd.concat([x.iloc[base_nb:base_nb + 1], x.iloc[base_nb + 2:base_nb + 3], x.dropna()]).transpose()
    
    recommand_inter.iloc[0, 2:] = recommand_inter.iloc[0,2:].map(lambda x : x.split(' ')[0])    
    
    
    recommand_inter.columns = range(len(recommand_inter.columns))
    recommand_inter.iloc[:, 0] = recommand_inter.iloc[:, 0] + str(2017 - i)
    
    recommand_inter.iloc[:, 1] = recommand_inter.iloc[:, 1].map(lambda x : x[0].upper() + x[1:], \
                                                                  na_action='ignore')

    propositions = x[x.iloc[:, 0].str.contains('No ').fillna(False)]
    propositions.columns = range(len(propositions.columns))
    
    if (propositions.iloc[:, 1].isnull()).all():
        if (2017 - i == 1997):
            propositions.iloc[:, 1] = propositions[0].map(lambda x : x.split(' ')[1])
            propositions.iloc[:, 0] = propositions[0].map(lambda x : x.split(' ')[0].rstrip())
        else:
            propositions.iloc[:, 1] = propositions[0].map(lambda x : x.split(':')[1][1:])
            propositions.iloc[:, 0] = propositions[0].map(lambda x : x.split(':')[0].rstrip())
    
    dico_no_propos = propositions.dropna(1).set_index(0).to_dict()
    
    if (dico_no_propos.get(1) == None) :
        recommand_inter.iloc[:, 1] = recommand_inter.iloc[:, 1].map(dico_no_propos)
    else :
        recommand_inter.iloc[:, 1] = recommand_inter.iloc[:, 1].map(dico_no_propos[1])
        
    recommand_columns = ['Date','Votation'] + list(recommand_inter.iloc[0][2:])
    recommand_inter.columns = recommand_columns
    recommand_inter['Date'] = recommand_inter['Date'].fillna(method='ffill')
    recommand_inter = recommand_inter.transpose()
    
    recommand_inter = recommand_inter.groupby(recommand_inter.index).first()
    recommand = recommand.join(recommand_inter, how='outer', lsuffix='l', rsuffix='r')

recommand = recommand.transpose()  

recommand.index = range(len(recommand.index))
recommand = recommand[~recommand.loc[:, 'Date'].str.contains('Parti 1').fillna(False)]
recommand.index = range(len(recommand.index))
recommand = recommand[~recommand['Votation'].isnull()]
#recommand = recommand[recommand_columns[1:]]

month_to_int = { \
    'janvier' : '01', \
    'février' : '02', \
    'févirer' : '02', \
    'mars' : '03', \
    'avril' : '04', \
    'mai' : '05', \
    'juin' : '06', \
    'juillet' : '07', \
    'aout' : '08', \
    'septembre' : '09', \
    'octobre' : '10', \
    'novembre' : '11', \
    'décembre' : '12' \
}

def good_format_month (x) :
    if '.' in x :
        split_x = x.split('.')
        return split_x[0].zfill(2) + '.' + split_x[1].zfill(2) + '.' + split_x[2][:4]
    else :
        return str(x.split(' ')[0]).zfill(2) + '.' +  month_to_int[str(x.split(' ')[1]) ] + '.' + x[-4:] 

recommand.loc[:, 'Date'] = recommand.loc[:, 'Date'].map(lambda x : good_format_month(x))

recommand = recommand.loc[:, recommand.columns.drop_duplicates() ]
parties = list(recommand.columns.drop_duplicates())
parties.remove('Date')
parties.remove('Votation')
parties.remove('Parti')

recommand = recommand.loc[:, ['Date', 'Votation'] + parties]
recommand

In [ ]:
parties

In [ ]:
recommand.columns.drop_duplicates()

In [ ]:
def translate_choice(x) :
    if (x == 1 or str(x) == 'oui') :
        return 1
    elif (x == 2 or str(x) == 'non'):
        return -1
    else :
        return 0

good_recommand = recommand.copy() 
good_recommand.loc[:, parties] = good_recommand.loc[:, parties].applymap(lambda x : translate_choice(x))
good_recommand

In [ ]:
good_recommand[good_recommand['Votation'].isnull()]

In [ ]:
dico_recom_vote = {}

for date1, group1 in df[df['Commune'] == 'Aeugst am Albis'].groupby('Date'):
    for date2, group2 in good_recommand.groupby('Date'):
        if (date1 == date2) :
            vot1 = list(group1['Votation']) 
            vot2 = list(group2['Votation'])
            while (len(vot1) > 0 and len(vot2) > 0) :
                max_v = -1
                max_match = [0, 0]
                for elem1  in vot1 :
                    for elem2 in  vot2 :
                        current_v = SequenceMatcher(None, elem1, elem2).ratio()
                        if(current_v > max_v) :
                            max_v = current_v
                            max_match=[elem1, elem2]
                dico_recom_vote[max_match[0]] = max_match[1]
                dico_recom_vote[max_match[1]] = max_match[0]
                vot1.remove(max_match[0])
                vot2.remove(max_match[1])
            
dico_recom_vote

In [ ]:
recommand_to_join = good_recommand.copy()
recommand_to_join['Votation'] = recommand_to_join['Votation'].map(dico_recom_vote)
recommand_to_join.head()

In [ ]:
switzerland_coord = [46.765213, 8.252444]
town_geo_path = r'data/switzerland_borders/admin_level_8.geojson'
geo_json_data = json.load(open(town_geo_path, encoding="utf8"))
commune = [x['name'] for x in geo_json_data['features']]

to_map = df.merge(recommand_to_join.loc[:, ['Votation'] + parties], on='Votation')
to_map = to_map[to_map['Commune'].isin(commune)]
to_map.head()

In [ ]:
for parti in parties :
    current_to_map = to_map.loc[:, ['Commune', 'Oui en %', parti]]
    current_to_map = current_to_map[current_to_map[parti] != 0]
    current_to_map['Agreement'] = current_to_map[['Oui en %', parti]] \
        .apply(lambda x : x['Oui en %'] if x[parti] == 1 else 100 - x['Oui en %'], axis=1)
    
    current_to_map = current_to_map.groupby('Commune', as_index=False).mean()
    
    print(current_to_map[current_to_map["Agreement"] == 50])
    
    map1 = folium.Map(location=switzerland_coord, zoom_start=8)
    map1.choropleth(geo_data = geo_json_data, \
                    data = current_to_map, \
                    columns = ['Commune', 'Agreement'], \
                    key_on = 'feature.name', \
                    fill_color = 'RdYlGn', \
                    fill_opacity = 0.7, \
                    line_opacity = 0.2, \
                    legend_name = 'Agreement in % with ' + parti, \
                    threshold_scale = [0, 50, 50, 50, 50, 100])
    
    map1.save('map ' + parti + '.html')
    print(map1)